# 나이브 베이즈 (naive bayes)
- 나이브 베이즈는 서로 조건부 독립(conditional independence)인 피처를 가정하고, 베이즈 이론을 기반으로 하는 머신러닝 알고리즘이다. 
- 확률 변수 Y가 주어졌을 때, 확률 변수 X1과 X2가 조건부 록립이면 아래 식을 만족한다 
![naive](img/Naive.png)

In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# 데이터 불러오기
raw_wine = datasets.load_wine()

In [3]:
# 피쳐, 타겟 데이터 지정
X = raw_wine.data
y = raw_wine.target

In [4]:
# 트레이닝/테스트 데이터 분할
X_tn, X_te, y_tn, y_te=train_test_split(X,y,random_state=0)

In [5]:
#데이터 표준화
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std  = std_scale.transform(X_te)

In [6]:
# 나이브 베이즈 학습
clf_gnb = GaussianNB()
clf_gnb.fit(X_tn_std, y_tn)

GaussianNB()

In [7]:
# 예측
pred_gnb = clf_gnb.predict(X_te_std)
print(pred_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 0 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [8]:
# 리콜 평가
recall = recall_score(y_te, pred_gnb, average='macro')
print(recall)

0.9523809523809524


In [9]:
# confusion matrix 확인 
conf_matrix = confusion_matrix(y_te, pred_gnb)
print(conf_matrix)

[[16  0  0]
 [ 2 18  1]
 [ 0  0  8]]


In [10]:
# 분류 레포트 확인
class_report = classification_report(y_te, pred_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.86      0.92        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.94      0.93      0.93        45



### Reference
###### 베이즈의 정리(Bayes' theorem)
- 나이브 베이즈 분류기를 이해하기 위해서는 우선 베이즈의 정리(Bayes' theorem)를 이해할 필요가 있습니다. 베이즈 정리는 조건부 확률을 계산하는 방법 중 하나입니다. 
- P(A)가 A가 일어날 확률, P(B)가 B가 일어날 확률, P(B|A)가 A가 일어나고나서 B가 일어날 확률, P(A|B)가 B가 일어나고나서 A가 일어날 확률이라고 해봅시다. 이때 P(B|A)를 쉽게 구할 수 있는 상황이라면, 아래와 같은 식을 통해  P(A|B)를 구할 수 있습니다. 

![naive](img/Bayes_theorem.png)

나이브 베이즈 분류기는 이러한 베이즈 정리를 이용하여 텍스트 분류를 수행합니다. 예를 들어서 나이브 베이즈 분류기를 통해서 스팸 메일 필터를 만들어본다고 합시다. 입력 텍스트(메일의 본문)이 주어졌을 때, 입력 텍스트가 정상 메일인지 스팸 메일인지 구분하기 위한 확률을 이와 같이 표현할 수 있습니다.

- P(정상 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 정상 메일일 확률
- P(스팸 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 스팸 메일일 확률

이를 베이즈의 정리에 따라서 식을 표현하면 이와 같습니다.

- P(정상 메일 | 입력 텍스트) = (P(입력 텍스트 | 정상 메일) × P(정상 메일)) / P(입력 텍스트)
- P(스팸 메일 | 입력 텍스트) = (P(입력 텍스트 | 스팸 메일) × P(스팸 메일)) / P(입력 텍스트)

입력 텍스트가 주어졌을 때, P(정상 메일 | 입력 텍스트)가 P(스팸 메일 | 입력 텍스트)보다 크다면 정상 메일이라고 볼 수 있으며, 그 반대라면 스팸 메일이라고 볼 수 있습니다. 그런데 두 확률 모두 식을 보면 P(입력 텍스트)를 분모로 하고 있음을 알 수 있습니다. 그렇기 때문에 분모를 양쪽에서 제거하여 식을 간소화합니다.

- P(정상 메일 | 입력 텍스트) = P(입력 텍스트 | 정상 메일) × P(정상 메일)
- P(스팸 메일 | 입력 텍스트) = P(입력 텍스트 | 스팸 메일) × P(스팸 메일)

입력 텍스트는 메일의 본문을 의미한다고 언급했습니다. 그런데 메일의 본문을 어떻게 나이브 베이즈 분류기의 입력으로 사용할 수 있을까요? 메일의 본문을 단어 토큰화하여 이 단어들을 나이브 베이즈의 분류기의 입력으로 사용합니다.

만약 메일의 본문에 있는 단어가 3개라고 가정해보겠습니다. 기본적으로 나이브 베이즈 분류기는 모든 단어가 독립적이라고 가정합니다. 메일의 본문에 있는 단어 3개를 w1,w2,w3라고 표현한다면 결국 나이브 베이즈 분류기의 정상 메일일 확률과 스팸 메일일 확률을 구하는 식은 아래와 같습니다.

- P(정상 메일 | 입력 텍스트) = P(w1 | 정상 메일) × P(w2 | 정상 메일) × P(w3 | 정상 메일) × P(정상 메일)
- P(스팸 메일 | 입력 텍스트) = P(w1 | 스팸 메일) × P(w2 | 스팸 메일) × P(w3 | 스팸 메일) × P(스팸 메일)

나이브 베이즈 분류기에서 토큰화 이전의 단어의 순서는 중요하지 않습니다. 즉, BoW와 같이 단어의 순서를 무시하고 오직 빈도수만을 고려합니다. 

[Naive Bayes Classifier](https://wikidocs.net/22892)